In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms

import spacy

spacy_eng = spacy.load("en_core_web_sm")


c:\coda\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\coda\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [56]:
df = pd.read_parquet("E:\\tamil statement analysis\\tamilmixsentiment-train.parquet")
df.iloc[:10]

,text,label
0,Trailer late ah parthavanga like podunga,0
1,Move pathutu vanthu trailer pakurvnga yaru,0
2,Puthupetai dhanush ah yarellam pathinga,0
3,"Dhanush oda character ,puthu sa erukay , mass ta",0
4,vera level ippa pesungada mokka nu thalaivaaaaaa,0
5,Thala mass . U1 bgm. Vera level,0
6,ivara pathta death vadi madiri irukku,1
7,0:28 90's kids reference Maadila irunthu kudi...,0
8,Aiyoo thala mass pannita thala,0
9,ENPT ku kaathu kadanthathellam oru nimishathul...,2


In [37]:
class Vocabulary():
    def __init__(self,freq_thres):
        self.itos = {0:'<PAD>',1:'<SOS>',2:'<EOS>',3:"<UNK>"}
        self.stoi =  {'<PAD>':0,'<SOS>':1,"<EOS>":2,"<UNK>":3}
        self.freq_thres = freq_thres
        
    def __len__(self):
        return len(self.itos)
    
    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]
    
    def build_vacabulary(self,sentence_list):
        frequencis = {}
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencis:
                    frequencis[word] = 1
                else:
                    frequencis[word] +=1
                    
                if frequencis[word] == self.freq_thres:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx+=1
                    
    def numericalize(self,text):
        tokenize_text = self.tokenizer_eng(text)
        return[
            self.stoi[token] if token in self.stoi else self.stoi['<UNK>']
            for token in tokenize_text
        ]

        



In [53]:
class dataset(Dataset):
    def __init__(self,df,fre_thresold=2):
        self.target  = df['label']
        self.caption = df['text']
        
        
        self.vocab = Vocabulary(fre_thresold)
        self.vocab.build_vacabulary(self.caption.tolist())
        
    def __len__(self):
        return len(self.target)
    def __getitem__(self,index):
        target=self.target[index]
        caption=self.caption[index]
        
        numericalize_caption = [self.vocab.stoi['<SOS>']]
        numericalize_caption+= self.vocab.numericalize(caption)
        numericalize_caption.append(self.vocab.stoi["<EOS>"])
        
        return torch.tensor(target),torch.tensor(numericalize_caption)
                
        

tensor([0])
tensor([[  1,   4, 233,  21, 950,   9,  60,   2]])
